In [3]:
import pandas as pd
from pandas.io.json import json_normalize
from pandasql import sqldf
dfsql = lambda q: sqldf(q, globals())

In [4]:
def flattencolumns(df1, cols):
    df = pd.concat([pd.DataFrame(df1[x].values.tolist()).add_prefix(x) for x in cols], axis=1)
    return pd.concat([df, df1.drop(cols, axis=1)], axis=1)

In [5]:
df = pd.read_json(
    './s3_sync/2017/07/10/06/riot-raw-games-ingestion-games-raw-3-2017-07-10-06-48-49-b235f850-0637-42d6-a9ad-1c89e755a6a7',
    lines=True,
    convert_axes=True,
    dtype={
        'gameCreation':'int64',
        'gameDuration':'int64',
        'gameId':'int64',
        'gameMode':'str',
        'gameType':'str',
        'gameVersion':'str',
        'mapId':'int64',
        'participantIdentities':'array',
        'participants':'array',
        'platformId':'array',
        'queueId':'int64',
        'seasonId':'int64',
        'teams':'str'
    }
)

In [6]:
df = pd.read_json(
    './s3_sync/2018/10/13/01/riot-raw-games-ingestion-games-raw-4-2018-10-13-01-22-45-f047db3c-2ca9-4ad4-bbb0-65f8f9dd442e',
    lines=True,
    convert_axes=True
)

In [7]:
df.dtypes

gameCreation              int64
gameDuration              int64
gameId                    int64
gameMode                 object
gameType                 object
gameVersion              object
mapId                     int64
participantIdentities    object
participants             object
platformId               object
queueId                   int64
seasonId                  int64
teams                    object
dtype: object

In [14]:
dfsql("""
select *
from fla
""")

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 7 - probably unsupported type. [SQL: 'INSERT INTO df ("gameCreation", "gameDuration", "gameId", "gameMode", "gameType", "gameVersion", "mapId", "participantIdentities", participants, "platformId", "queueId", "seasonId", teams) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: (1539390368227, 1487, 2885747863, 'CLASSIC', 'MATCHED_GAME', '8.20.249.1952', 11, [{'participantId': 1, 'player': {'platformId': 'NA1', 'accountId': 42212761, 'summonerName': 'I AFK or FEED I', 'summonerId': 27761252, 'currentPlatfo ... (2438 characters truncated) ...  47562521, 'currentPlatformId': 'NA1', 'currentAccountId': 210309937, 'matchHistoryUri': '/v1/stats/player_history/NA1/210309937', 'profileIcon': 0}}], [{'participantId': 1, 'teamId': 100, 'championId': 63, 'spell1Id': 11, 'spell2Id': 4, 'highestAchievedSeasonTier': 'SILVER', 'stats': {'participantId' ... (28197 characters truncated) ... ltas': {'10-20': 1258.9, '0-10': 369.

In [8]:
flattend = flattencolumns(df, ['participantIdentities'])
flattend

,participantIdentities0,participantIdentities1,participantIdentities2,participantIdentities3,participantIdentities4,participantIdentities5,participantIdentities6,participantIdentities7,participantIdentities8,participantIdentities9,...,gameId,gameMode,gameType,gameVersion,mapId,participants,platformId,queueId,seasonId,teams
0,"{'participantId': 1, 'player': {'platformId': ...","{'participantId': 2, 'player': {'platformId': ...","{'participantId': 3, 'player': {'platformId': ...","{'participantId': 4, 'player': {'platformId': ...","{'participantId': 5, 'player': {'platformId': ...","{'participantId': 6, 'player': {'platformId': ...","{'participantId': 7, 'player': {'platformId': ...","{'participantId': 8, 'player': {'platformId': ...","{'participantId': 9, 'player': {'platformId': ...","{'participantId': 10, 'player': {'platformId':...",...,2885747863,CLASSIC,MATCHED_GAME,8.20.249.1952,11,"[{'participantId': 1, 'teamId': 100, 'champion...",NA1,420,11,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F..."


In [9]:
def lateral_flatten(df, column_name):
    rows = []
    for i, row in df.iterrows():
        particpiant_identities = row.at['participantIdentities']
        lateral_pi = json_normalize(particpiant_identities)
        participants = row.at['participants']
        lateral_p = json_normalize(participants)
        teams = row.at['teams']
        lateral_teams = json_normalize(teams)
        
        merge1 = pd.merge(lateral_pi, lateral_p, on='participantId')
        merge2 = pd.merge(merge1, lateral_teams, on='teamId')
        
        for j, participant_row in merge2.iterrows():
            new = row.copy()
            
            new_row = pd.concat([new, participant_row])
            rows.append(new_row)

    return pd.DataFrame(rows)

In [10]:
lateral_flatten(df, 'foo').columns.values

array(['gameCreation', 'gameDuration', 'gameId', 'gameMode', 'gameType',
       'gameVersion', 'mapId', 'participantIdentities', 'participants',
       'platformId', 'queueId', 'seasonId', 'teams', 'participantId',
       'player.accountId', 'player.currentAccountId',
       'player.currentPlatformId', 'player.matchHistoryUri',
       'player.platformId', 'player.profileIcon', 'player.summonerId',
       'player.summonerName', 'championId', 'highestAchievedSeasonTier',
       'spell1Id', 'spell2Id', 'stats.assists', 'stats.champLevel',
       'stats.combatPlayerScore', 'stats.damageDealtToObjectives',
       'stats.damageDealtToTurrets', 'stats.damageSelfMitigated',
       'stats.deaths', 'stats.doubleKills', 'stats.firstBloodAssist',
       'stats.firstBloodKill', 'stats.firstInhibitorAssist',
       'stats.firstInhibitorKill', 'stats.firstTowerAssist',
       'stats.firstTowerKill', 'stats.goldEarned', 'stats.goldSpent',
       'stats.inhibitorKills', 'stats.item0', 'stats.item1',
  

In [11]:
test = df[["gameCreation","gameDuration","gameId","gameMode","gameType","gameVersion","mapId","participantIdentities"]]

In [12]:
df['participantIdentities'][0]

[{'participantId': 1,
  'player': {'platformId': 'NA1',
   'accountId': 42212761,
   'summonerName': 'I AFK or FEED I',
   'summonerId': 27761252,
   'currentPlatformId': 'NA1',
   'currentAccountId': 42212761,
   'matchHistoryUri': '/v1/stats/player_history/NA1/42212761',
   'profileIcon': 691}},
 {'participantId': 2,
  'player': {'platformId': 'NA1',
   'accountId': 226903062,
   'summonerName': 'GhostieRoast',
   'summonerId': 64951811,
   'currentPlatformId': 'NA1',
   'currentAccountId': 226903062,
   'matchHistoryUri': '/v1/stats/player_history/NA1/226903062',
   'profileIcon': 1452}},
 {'participantId': 3,
  'player': {'platformId': 'NA1',
   'accountId': 226161792,
   'summonerName': 'Teriyaki God',
   'summonerId': 62661505,
   'currentPlatformId': 'NA1',
   'currentAccountId': 226161792,
   'matchHistoryUri': '/v1/stats/player_history/NA1/226161792',
   'profileIcon': 1450}},
 {'participantId': 4,
  'player': {'platformId': 'NA1',
   'accountId': 234933660,
   'summonerName':

In [13]:
dfsql("""
select *
from df
""")

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 7 - probably unsupported type. [SQL: 'INSERT INTO df ("gameCreation", "gameDuration", "gameId", "gameMode", "gameType", "gameVersion", "mapId", "participantIdentities", participants, "platformId", "queueId", "seasonId", teams) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: (1539390368227, 1487, 2885747863, 'CLASSIC', 'MATCHED_GAME', '8.20.249.1952', 11, [{'participantId': 1, 'player': {'platformId': 'NA1', 'accountId': 42212761, 'summonerName': 'I AFK or FEED I', 'summonerId': 27761252, 'currentPlatfo ... (2438 characters truncated) ...  47562521, 'currentPlatformId': 'NA1', 'currentAccountId': 210309937, 'matchHistoryUri': '/v1/stats/player_history/NA1/210309937', 'profileIcon': 0}}], [{'participantId': 1, 'teamId': 100, 'championId': 63, 'spell1Id': 11, 'spell2Id': 4, 'highestAchievedSeasonTier': 'SILVER', 'stats': {'participantId' ... (28197 characters truncated) ... ltas': {'10-20': 1258.9, '0-10': 369.

In [4]:
df.to_parquet("./s3_parquet/foo.parquet", engine="fastparquet", compression='snappy')

In [7]:
df_normalized = json_normalize(
    df    
)

AttributeError: 'str' object has no attribute 'values'

In [ ]:
pd.read_parquet("./s3_parquet/foo.parquet", engine="fastparquet").dtypes

In [12]:
one_row_table = df.head(n=1)

In [52]:
df

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,teams
0,1499665649596,1567,2544823989,CLASSIC,MATCHED_GAME,7.13.192.6794,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",NA1,400,8,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
1,1499663117674,1785,2544810915,CLASSIC,MATCHED_GAME,7.13.192.6794,11,"[{'participantId': 1}, {'participantId': 2}, {...","[{'participantId': 1, 'teamId': 100, 'champion...",NA1,400,8,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
2,1499660524114,2048,2544655056,CLASSIC,MATCHED_GAME,7.13.192.6794,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",NA1,400,8,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
3,1499572402641,1514,2543872569,CLASSIC,MATCHED_GAME,7.13.192.6794,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",NA1,420,8,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F..."
4,1499556741221,2476,2543685823,CLASSIC,MATCHED_GAME,7.13.192.6794,11,"[{'participantId': 1}, {'participantId': 2}, {...","[{'participantId': 1, 'teamId': 100, 'champion...",NA1,400,8,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F..."
5,1499554266343,2059,2543681846,CLASSIC,MATCHED_GAME,7.13.192.6794,11,"[{'participantId': 1}, {'participantId': 2}, {...","[{'participantId': 1, 'teamId': 100, 'champion...",NA1,400,8,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
6,1499552061048,1545,2543636860,CLASSIC,MATCHED_GAME,7.13.192.6794,11,"[{'participantId': 1}, {'participantId': 2, 'p...","[{'participantId': 1, 'teamId': 100, 'champion...",NA1,400,8,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
7,1499550279174,1306,2543625615,CLASSIC,MATCHED_GAME,7.13.192.6794,11,"[{'participantId': 1}, {'participantId': 2}, {...","[{'participantId': 1, 'teamId': 100, 'champion...",NA1,400,8,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
8,1499547707569,1714,2543609034,CLASSIC,MATCHED_GAME,7.13.192.6794,11,"[{'participantId': 1}, {'participantId': 2}, {...","[{'participantId': 1, 'teamId': 100, 'champion...",NA1,400,8,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
9,1499498878935,2211,2543383072,CLASSIC,MATCHED_GAME,7.13.192.6794,11,"[{'participantId': 1}, {'participantId': 2, 'p...","[{'participantId': 1, 'teamId': 100, 'champion...",NA1,400,8,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F..."


In [1]:
test = flattencolumns(df, ['participantIdentities', 'teams', 'participants'])
test = flattencolumns(df, ['bans', 'masteries'])

NameError: name 'flattencolumns' is not defined

In [61]:
test.to_parquet("./s3_parquet/foo.parquet", engine="pyarrow", compression='snappy')

ImportError: pyarrow is required for parquet support

you can install via conda
conda install pyarrow -c conda-forge

or via pip
pip install -U pyarrow
